# Merging state/county/city polygons with party affiliation and landvote data

In [ ]:
import ibis
from ibis import _

import streamlit as st
import ibis.expr.datatypes as dt  # Make sure to import the necessary module

conn = ibis.duckdb.connect(extensions=["spatial"])

landvote_url = "https://huggingface.co/datasets/boettiger-lab/landvote/resolve/main/landvote_polygons.parquet"
# party_url = "https://huggingface.co/datasets/boettiger-lab/landvote/resolve/main/party_polygons.parquet"
party_url = "https://huggingface.co/datasets/boettiger-lab/landvote/resolve/main/party_polygons_all.parquet"



In [ ]:
landvote = (conn
            .read_parquet(landvote_url)
            .cast({"geometry": "geometry"})
            .mutate(county = _.county.upper())
            .mutate(municipal = _.municipal.upper())
            .mutate(elect_year = _.year - _.year % 4) # get most recent election year 
            .cast({"municipal": "string","county":"string"})
            .mutate(municipal=ibis.case()
                 .when(_.jurisdiction.isin(['State','County']), ibis.literal("-")) 
                 .else_(_.municipal) 
                 .end()
                 )
            .mutate(county=ibis.case()
                 .when(_.jurisdiction.isin(['State']), ibis.literal("-"))
                 .else_(ibis.case()
                         .when(_.county.endswith('COUNTY'), _.county)
                         .else_(_.county + ' COUNTY')
                         .end())
                 .end())
           )

party = (conn
        .read_parquet(party_url)
        .cast({"geometry": "geometry","municipal":"string"})
        .mutate(municipal=ibis.case()
                .when(_.jurisdiction.isin(['State','County']), ibis.literal("-")) 
                .else_(_.municipal) 
                .end()
                )
        .mutate(county=ibis.case()
                .when(_.jurisdiction.isin(['State']), ibis.literal("-")) 
                .else_(_.county) 
                .end()
                )
        )

In [ ]:
votes = (landvote
        .join(party,["state","county","municipal","jurisdiction","geometry", _.elect_year == party["year"]],how = "left")
        .drop('elect_year','state_right','county_right','municipal_right','year_right',"geometry_right","jurisdiction_right")
        .mutate(municipal=ibis.case()
                .when(_.municipal == ibis.literal("-"), None) 
                .else_(_.municipal) 
                .end()
                )
        .mutate(county=ibis.case()
                .when(_.county == ibis.literal("-"), None) 
                .else_(_.county) 
                .end()
                )
        )

# Make PMTiles. Each jurisdiction type is its own layer

In [ ]:
import subprocess
import os
from huggingface_hub import HfApi, login
import streamlit as st

login(st.secrets["HF_TOKEN"])
# api = HfApi(add_to_git_credential=False)
api = HfApi()

def hf_upload(file, repo_id):
    info = api.upload_file(
            path_or_fileobj=file,
            path_in_repo=file,
            repo_id=repo_id,
            repo_type="dataset",
        )
def generate_pmtiles(input_file, input_file2, input_file3, output_file, max_zoom=12):
    # Ensure Tippecanoe is installed
    if subprocess.call(["which", "tippecanoe"], stdout=subprocess.DEVNULL) != 0:
        raise RuntimeError("Tippecanoe is not installed or not in PATH")

    # Construct the Tippecanoe command
    command = [
        "tippecanoe",
        "-o", output_file,
        "-zg",
        "--extend-zooms-if-still-dropping",
        "--force",
        "--projection", "EPSG:4326",  
        "-L","state:"+input_file,
        "-L","county:"+input_file2,
        "-L","municipal:"+input_file3
    ]
    # Run Tippecanoe
    try:
        subprocess.run(command, check=True)
        print(f"Successfully generated PMTiles file: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error running Tippecanoe: {e}")



In [ ]:
gdf_state = votes.filter(_.jurisdiction == 'State').execute().set_crs("EPSG:4326")
gdf_state.to_file("votes_state.geojson")

gdf_county = votes.filter(_.jurisdiction == 'County').execute().set_crs("EPSG:4326")
gdf_county.to_file("votes_county.geojson")

gdf_city = votes.filter(_.jurisdiction == 'Municipal').execute().set_crs("EPSG:4326")
gdf_city.to_file("votes_municipal.geojson")

generate_pmtiles("votes_state.geojson", "votes_county.geojson","votes_municipal.geojson", "votes.pmtiles")
hf_upload("votes.pmtiles", "boettiger-lab/landvote")

In [ ]:
# save as parquet
votes.execute().set_crs("EPSG:4326").to_parquet("votes.parquet")
hf_upload("votes.parquet", "boettiger-lab/landvote")
